# 033: Prophet & Modern Time Series Methods 🚀

## Learning Objectives
- Master **Facebook Prophet** for automatic time series forecasting
- Understand **trend changepoints** and automatic detection
- Model **multiple seasonality** (daily + weekly + yearly)
- Handle **holidays and special events** in forecasting
- Apply **TimesFM** (Google's foundation model for time series)
- Compare **Prophet vs ARIMA vs Exponential Smoothing**

---

## 🔄 Prophet Forecasting Pipeline

```mermaid
graph TD
    A[Time Series Data] --> B[Prophet Model]
    B --> C{Components}
    
    C --> D[Trend: g t]
    C --> E[Seasonality: s t]
    C --> F[Holidays: h t]
    
    D --> G[Linear/Logistic Growth]
    D --> H[Changepoint Detection]
    
    E --> I[Fourier Series]
    E --> J[Multiple Periods]
    
    F --> K[Custom Events]
    
    G --> L[Forecast: y = g t + s t + h t + ε]
    H --> L
    I --> L
    J --> L
    K --> L
    
    L --> M[Prediction Intervals]
```

---

## 📊 Prophet vs Traditional Methods

| **Aspect** | **Prophet** | **ARIMA** | **Exponential Smoothing** |
|------------|-------------|-----------|---------------------------|
| **Seasonality** | Automatic (Fourier series, multiple) | Manual (SARIMA) | Manual (Holt-Winters) |
| **Trend** | Automatic changepoints | Differencing | Linear/damped |
| **Holidays** | Built-in support | Manual dummies | Not supported |
| **Missing data** | Handles automatically | Requires imputation | Requires imputation |
| **Interpretability** | High (additive components) | Moderate | High |
| **Tuning** | Minimal (good defaults) | Heavy (p,d,q selection) | Moderate (α,β,γ) |
| **Speed** | Fast (Stan backend) | Fast | Very fast |
| **Best For** | Business forecasting, multiple seasonality | Complex autocorrelations | Short-term, simple patterns |

---

## 🎯 Key Concepts

### 1. **Prophet Model Equation**

Prophet decomposes time series into additive components:

$$
y(t) = g(t) + s(t) + h(t) + \epsilon_t
$$

Where:
- $g(t)$ = **Trend** (piecewise linear or logistic growth with changepoints)
- $s(t)$ = **Seasonality** (Fourier series for periodic patterns)
- $h(t)$ = **Holidays** (custom events with windows)
- $\epsilon_t$ = **Error** (normally distributed noise)

---

### 2. **Trend with Changepoints**

Prophet uses **piecewise linear trend** with automatic changepoint detection:

$$
g(t) = (k + \mathbf{a}(t)^T \boldsymbol{\delta}) t + (m + \mathbf{a}(t)^T \boldsymbol{\gamma})
$$

Where:
- $k$ = base growth rate
- $\boldsymbol{\delta}$ = rate adjustments at changepoints
- $m$ = offset parameter
- $\mathbf{a}(t)$ = indicator vector (1 if t after changepoint, 0 otherwise)

**Changepoint detection:**
- By default, Prophet places 25 potential changepoints uniformly in first 80% of data
- Uses **L1 regularization** to select significant changepoints (sparse $\boldsymbol{\delta}$)
- `changepoint_prior_scale` controls flexibility (default 0.05, higher = more flexible)

---

### 3. **Seasonality via Fourier Series**

Prophet models seasonality using **Fourier series** (sum of sines/cosines):

$$
s(t) = \sum_{n=1}^{N} \left( a_n \cos\left(\frac{2\pi nt}{P}\right) + b_n \sin\left(\frac{2\pi nt}{P}\right) \right)
$$

Where:
- $P$ = period (365.25 for yearly, 7 for weekly)
- $N$ = number of Fourier terms (more terms = more flexible)
  - Yearly: N=10 (default)
  - Weekly: N=3 (default)
  - Daily: N=4 (default)

**Multiple seasonality:** Prophet can model yearly + weekly + daily simultaneously (additive)

---

### 4. **Holidays and Events**

Prophet handles special events with indicator variables:

$$
h(t) = \sum_{i=1}^{L} \kappa_i \cdot \mathbb{1}_{[D_i - w, D_i + w]}(t)
$$

Where:
- $D_i$ = date of holiday i
- $w$ = window around holiday (e.g., ±3 days for holiday week)
- $\kappa_i$ = impact of holiday i (learned from data)

**Built-in holidays:** Prophet includes major holidays for 100+ countries (US, UK, etc.)

---

## 🔬 Post-Silicon Validation Application

### **Long-Term Yield Forecasting**
- **Problem:** Forecast weekly yield 26 weeks ahead accounting for:
  - Process improvements (non-linear trend with changepoints)
  - Quarterly PM cycles (seasonal)
  - Major equipment upgrades (holidays/events)
- **Prophet Solution**: Automatic changepoint detection captures ramp-up phases, quarterly seasonality built-in, custom events for upgrades
- **Business Value**: $10M+ savings via accurate capacity planning for 6-month horizon

---

### 📝 What's Happening in This Code?

**Purpose:** Import Prophet library for automatic time series forecasting

**Key Points:**
- **Prophet**: Facebook's open-source forecasting tool (Stan backend for Bayesian inference)
- **pandas**: Required data format (DataFrame with 'ds' and 'y' columns)
- **matplotlib**: For component visualization (trend, seasonality, holidays)
- **Prophet philosophy**: "Make good forecasts accessible to non-experts" (automatic everything)

**Why This Matters:** Prophet requires ZERO manual parameter tuning (unlike ARIMA's p,d,q), making it ideal for production systems with 100+ time series to forecast.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

### 📝 Basic Prophet Forecasting

**Purpose:** Fit Prophet model on yield data with automatic trend + seasonality detection

**Key Points:**
- **Data format**: Must have 'ds' (datetime) and 'y' (value) columns
- **fit()**: Estimates trend, seasonality, holidays using Stan (Bayesian inference)
- **make_future_dataframe()**: Creates future dates for forecasting
- **predict()**: Returns forecast with 'yhat' (point estimate), 'yhat_lower', 'yhat_upper' (80% interval)
- **Automatic features**: Detects changepoints, adds yearly + weekly seasonality by default

**Post-Silicon Example:** Weekly yield with process improvement changepoints (80% → 95% over 2 years)

In [ ]:
# Generate synthetic yield data with changepoints (process improvements)
np.random.seed(42)
dates = pd.date_range('2021-01-01', periods=104, freq='W')  # 2 years weekly

# Piecewise linear trend with changepoints at week 30 (process upgrade) and week 70 (another upgrade)
trend = np.concatenate([
    np.linspace(80, 85, 30),   # Phase 1: slow improvement
    np.linspace(85, 92, 40),   # Phase 2: rapid improvement (upgrade)
    np.linspace(92, 95, 34)    # Phase 3: mature process
])

# Add weekly seasonality (slight drop mid-week due to maintenance)
weekly_pattern = np.tile([0, -0.5, -1, -0.5, 0, 0.5, 0.5], 15)[:104]

# Combine with noise
yield_data = trend + weekly_pattern + 1.5 * np.random.randn(104)

# Create Prophet-compatible DataFrame
df = pd.DataFrame({'ds': dates, 'y': yield_data})

# Fit Prophet model (default settings)
model = Prophet()
model.fit(df)

# Forecast next 26 weeks (6 months)
future = model.make_future_dataframe(periods=26, freq='W')
forecast = model.predict(future)

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Main forecast plot
axes[0].plot(df['ds'], df['y'], 'ko', label='Actual Yield %', alpha=0.6)
axes[0].plot(forecast['ds'], forecast['yhat'], 'b-', label='Prophet Forecast', linewidth=2)
axes[0].fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], 
                      alpha=0.3, color='blue', label='80% Confidence Interval')
axes[0].axvline(dates[-1], color='red', linestyle='--', label='Forecast Start', linewidth=2)
axes[0].set_title('Prophet Forecast - Yield with Automatic Changepoint Detection', fontsize=14)
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Yield %')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Changepoint visualization
changepoints = model.changepoints
for cp in changepoints[-5:]:  # Show last 5 changepoints
    axes[0].axvline(cp, color='green', linestyle=':', alpha=0.5, linewidth=1)

# Component plot (trend)
axes[1].plot(forecast['ds'], forecast['trend'], 'g-', label='Trend Component', linewidth=2)
for cp in changepoints[-5:]:
    axes[1].axvline(cp, color='red', linestyle=':', alpha=0.5, linewidth=1, label='Changepoint' if cp == changepoints[-5] else '')
axes[1].set_title('Extracted Trend with Changepoints', fontsize=14)
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Yield %')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print forecast summary
print("Prophet Forecast Summary:")
print(f"  Total changepoints detected: {len(model.changepoints)}")
print(f"  Last changepoint: {model.changepoints[-1].date()}")
print(f"\nForecast (next 4 weeks):")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(4).to_string(index=False))
print(f"\nProjected yield 26 weeks ahead: {forecast['yhat'].iloc[-1]:.2f}% ± {(forecast['yhat_upper'].iloc[-1] - forecast['yhat'].iloc[-1]):.2f}%")

### 📝 Multiple Seasonality and Custom Holidays

**Purpose:** Model multiple seasonal patterns + custom events (equipment upgrades)

**Key Points:**
- **add_seasonality()**: Add custom seasonal patterns (e.g., quarterly PM cycles)
- **Fourier order**: Higher order = more flexible seasonality (default: yearly=10, weekly=3)
- **add_country_holidays()**: Built-in holidays for 100+ countries
- **Custom holidays**: Define events as DataFrame with 'ds' (date) and 'holiday' (name)

**Post-Silicon Example:** Yield with weekly + quarterly PM seasonality + major equipment upgrade events

In [ ]:
# Generate data with multiple seasonality
np.random.seed(42)
dates = pd.date_range('2021-01-01', periods=104, freq='W')

# Base trend
trend = 85 + 0.1 * np.arange(104)

# Weekly seasonality (maintenance mid-week)
weekly = np.tile([0, -0.8, -1.2, -0.8, 0, 0.5, 0.5], 15)[:104]

# Quarterly seasonality (PM cycles every 13 weeks)
quarterly = -3 * np.sin(2 * np.pi * np.arange(104) / 13)

# Combine
yield_multi = trend + weekly + quarterly + 1.2 * np.random.randn(104)

# Define custom holidays (equipment upgrades causing temporary yield drop)
upgrades = pd.DataFrame({
    'holiday': 'equipment_upgrade',
    'ds': pd.to_datetime(['2021-06-15', '2022-03-01', '2022-09-15']),
    'lower_window': -7,   # 1 week before (prep)
    'upper_window': 7     # 1 week after (stabilization)
})

df_multi = pd.DataFrame({'ds': dates, 'y': yield_multi})

# Fit Prophet with custom seasonality + holidays
model_multi = Prophet()

# Add quarterly seasonality (in addition to default yearly + weekly)
model_multi.add_seasonality(
    name='quarterly',
    period=13,           # 13 weeks (quarter)
    fourier_order=5      # Flexibility
)

# Add custom holidays
model_multi.holidays = upgrades

model_multi.fit(df_multi)

# Forecast
future_multi = model_multi.make_future_dataframe(periods=26, freq='W')
forecast_multi = model_multi.predict(future_multi)

# Visualize components
fig = model_multi.plot_components(forecast_multi, figsize=(14, 10))
plt.tight_layout()
plt.show()

# Component breakdown
print("Prophet Component Analysis:")
print(f"\n1. Trend: Overall {'increasing' if forecast_multi['trend'].iloc[-1] > forecast_multi['trend'].iloc[0] else 'decreasing'}")
print(f"   Start: {forecast_multi['trend'].iloc[0]:.2f}%, End: {forecast_multi['trend'].iloc[-1]:.2f}%")

print(f"\n2. Weekly Seasonality:")
weekly_effect = forecast_multi['weekly'].iloc[:7]
print(f"   Max impact: {weekly_effect.max():.2f}% (day {weekly_effect.idxmax() % 7})")
print(f"   Min impact: {weekly_effect.min():.2f}% (day {weekly_effect.idxmin() % 7})")

print(f"\n3. Quarterly Seasonality:")
quarterly_effect = forecast_multi['quarterly'].iloc[:13]
print(f"   Amplitude: ±{quarterly_effect.abs().max():.2f}%")

print(f"\n4. Equipment Upgrade Impact:")
print(f"   Number of upgrades: {len(upgrades)}")
print(f"   Average impact: {forecast_multi['equipment_upgrade'].dropna().mean():.2f}% (temporary drop)")

print(f"\nOverall Forecast Range: {forecast_multi['yhat'].min():.2f}% - {forecast_multi['yhat'].max():.2f}%")

### 📝 Prophet vs ARIMA vs Exponential Smoothing Comparison

**Purpose:** Compare three forecasting methods on same dataset (yield with trend + seasonality)

**Key Points:**
- **ARIMA**: Requires stationarity, manual (p,d,q) tuning, good for complex autocorrelations
- **Exponential Smoothing**: Fast, no stationarity, fixed seasonal patterns
- **Prophet**: Automatic, handles missing data, flexible seasonality, interpretable components
- **Evaluation**: RMSE, MAE on 12-week test set

**Post-Silicon Example:** Which method best forecasts yield with quarterly PM cycles?

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Use previous yield data with quarterly PM pattern
train_df = df_multi[:-12]  # Train on first 92 weeks
test_df = df_multi[-12:]   # Test on last 12 weeks

# --- Prophet ---
prophet_comp = Prophet()
prophet_comp.add_seasonality(name='quarterly', period=13, fourier_order=5)
prophet_comp.fit(train_df)
prophet_forecast_comp = prophet_comp.predict(model_multi.make_future_dataframe(periods=12, freq='W'))
prophet_pred = prophet_forecast_comp['yhat'].iloc[-12:].values

# --- ARIMA ---
# Use SARIMA(1,1,1)(1,0,1,13) for quarterly seasonality
arima_model = ARIMA(train_df['y'], order=(1,1,1), seasonal_order=(1,0,1,13))
arima_fit = arima_model.fit()
arima_pred = arima_fit.forecast(steps=12)

# --- Exponential Smoothing (Holt-Winters) ---
hw_model = ExponentialSmoothing(
    train_df['y'], 
    trend='add', 
    seasonal='add', 
    seasonal_periods=13
).fit()
hw_pred = hw_model.forecast(steps=12)

# Calculate metrics
prophet_rmse = np.sqrt(mean_squared_error(test_df['y'], prophet_pred))
prophet_mae = mean_absolute_error(test_df['y'], prophet_pred)

arima_rmse = np.sqrt(mean_squared_error(test_df['y'], arima_pred))
arima_mae = mean_absolute_error(test_df['y'], arima_pred)

hw_rmse = np.sqrt(mean_squared_error(test_df['y'], hw_pred))
hw_mae = mean_absolute_error(test_df['y'], hw_pred)

# Visualize comparison
plt.figure(figsize=(14, 6))
plt.plot(train_df['ds'], train_df['y'], 'ko', label='Training Data', alpha=0.5)
plt.plot(test_df['ds'], test_df['y'], 'ro', label='Actual (Test)', markersize=8, alpha=0.7)
plt.plot(test_df['ds'], prophet_pred, 'b-', label=f'Prophet (RMSE: {prophet_rmse:.2f})', linewidth=2, marker='s')
plt.plot(test_df['ds'], arima_pred, 'g--', label=f'ARIMA (RMSE: {arima_rmse:.2f})', linewidth=2, marker='o')
plt.plot(test_df['ds'], hw_pred, 'm-.', label=f'Holt-Winters (RMSE: {hw_rmse:.2f})', linewidth=2, marker='^')
plt.axvline(train_df['ds'].iloc[-1], color='black', linestyle=':', label='Train/Test Split', linewidth=2)
plt.title('Forecasting Method Comparison - 12-Week Test Set', fontsize=14)
plt.xlabel('Date')
plt.ylabel('Yield %')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Results table
print("Forecasting Method Comparison (12-Week Test Set):\n")
print(f"{'Method':<20} {'RMSE':<10} {'MAE':<10} {'Best?':<10}")
print("="*50)
print(f"{'Prophet':<20} {prophet_rmse:<10.3f} {prophet_mae:<10.3f} {'✅' if prophet_rmse == min(prophet_rmse, arima_rmse, hw_rmse) else ''}")
print(f"{'ARIMA':<20} {arima_rmse:<10.3f} {arima_mae:<10.3f} {'✅' if arima_rmse == min(prophet_rmse, arima_rmse, hw_rmse) else ''}")
print(f"{'Holt-Winters':<20} {hw_rmse:<10.3f} {hw_mae:<10.3f} {'✅' if hw_rmse == min(prophet_rmse, arima_rmse, hw_rmse) else ''}")

print("\n📊 Key Insights:")
if prophet_rmse < arima_rmse and prophet_rmse < hw_rmse:
    print("  - Prophet wins: Best for automatic changepoint detection + flexible seasonality")
elif arima_rmse < prophet_rmse and arima_rmse < hw_rmse:
    print("  - ARIMA wins: Best for complex autocorrelation structures (but requires manual tuning)")
else:
    print("  - Holt-Winters wins: Fast and effective for simple seasonal patterns")

print(f"  - Prophet advantage: Handles missing data, no stationarity requirement")
print(f"  - ARIMA advantage: Theoretical foundation, good for short-memory processes")
print(f"  - Holt-Winters advantage: Fastest training, interpretable parameters (α, β, γ)")

## 🔧 Prophet vs Traditional Methods

Compare Prophet with ARIMA and Exponential Smoothing:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Comparison table
comparison = pd.DataFrame({
    'Aspect': [
        'Trend Handling',
        'Seasonality',
        'Holidays/Events',
        'Missing Data',
        'Outliers',
        'Multiple Seasonalities',
        'Changepoints',
        'Uncertainty Intervals',
        'Ease of Use',
        'Interpretability',
        'Speed',
        'Best For'
    ],
    'Prophet': [
        'Excellent (piecewise linear/logistic)',
        'Excellent (Fourier series)',
        'Excellent (built-in)',
        'Excellent (automatic handling)',
        'Good (robust to outliers)',
        'Yes (daily, weekly, yearly)',
        'Automatic detection',
        'Yes (uncertainty intervals)',
        'Very Easy',
        'High (decomposable)',
        'Fast',
        'Business forecasting with holidays'
    ],
    'ARIMA': [
        'Good (differencing)',
        'Good (seasonal ARIMA)',
        'Manual (exogenous variables)',
        'Poor (requires complete data)',
        'Poor (sensitive)',
        'No (single seasonality)',
        'Manual parameter tuning',
        'Yes (statistical)',
        'Hard (p,d,q selection)',
        'Medium',
        'Medium',
        'Stationary univariate series'
    ],
    'Exp Smoothing': [
        'Good (Holt method)',
        'Good (Holt-Winters)',
        'No',
        'Poor (requires complete data)',
        'Poor (sensitive)',
        'No (single seasonality)',
        'No',
        'Yes (bootstrap)',
        'Easy',
        'High (smoothing constants)',
        'Fast',
        'Short-term forecasts, simple patterns'
    ]
})

print('\n📊 Prophet vs Traditional Time Series Methods:\n')
print(comparison.to_string(index=False))

# Scoring visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Score categories
categories = ['Ease of Use', 'Seasonality', 'Missing Data', 'Holidays', 'Outliers']
prophet_scores = [5, 5, 5, 5, 4]  # Out of 5
arima_scores = [2, 4, 1, 1, 1]
exp_scores = [4, 4, 1, 0, 1]

x = range(len(categories))
width = 0.25

ax1.bar([i - width for i in x], prophet_scores, width, label='Prophet', color='#4285F4', edgecolor='black', linewidth=1.5)
ax1.bar(x, arima_scores, width, label='ARIMA', color='#EA4335', edgecolor='black', linewidth=1.5)
ax1.bar([i + width for i in x], exp_scores, width, label='Exp Smoothing', color='#FBBC04', edgecolor='black', linewidth=1.5)

ax1.set_xticks(x)
ax1.set_xticklabels(categories, rotation=45, ha='right')
ax1.set_ylabel('Score (0-5)', fontsize=12, fontweight='bold')
ax1.set_title('Method Comparison (Higher is Better)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.set_ylim(0, 5.5)
ax1.grid(axis='y', alpha=0.3)

# Use case decision tree
ax2.axis('off')
decision_guide = [
    "🎯 Which Method to Choose?\n",
    "✅ Choose Prophet when:",
    "   • Business forecasting with holidays/events",
    "   • Missing data or outliers present",
    "   • Multiple seasonalities (daily + yearly)",
    "   • Need interpretable components",
    "   • Fast prototyping required\n",
    "✅ Choose ARIMA when:",
    "   • Stationary univariate time series",
    "   • Statistical rigor required",
    "   • Single seasonality pattern",
    "   • Academic/research context\n",
    "✅ Choose Exponential Smoothing when:",
    "   • Short-term forecasts (<30 days)",
    "   • Simple trend/seasonality patterns",
    "   • Computational speed critical",
    "   • Baseline comparison needed\n",
    "💡 Modern Recommendation: Start with Prophet,",
    "   compare with ARIMA/ES for validation."
]

ax2.text(0.05, 0.95, '\n'.join(decision_guide), transform=ax2.transAxes,
        fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.tight_layout()
plt.savefig('prophet_vs_traditional.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n✅ Prophet advantages:')
print('  1. Automatic handling of missing data and outliers')
print('  2. Built-in holiday/event effects')
print('  3. Multiple seasonalities (daily, weekly, yearly)')
print('  4. Intuitive parameter names (growth, changepoint_prior_scale)')
print('  5. Fast and scalable')
print('\n💡 Prophet = "Time Series for Humans"')

## 🏭 Post-Silicon Validation Applications

Real-world use cases for Prophet in semiconductor testing:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Use cases table
use_cases = pd.DataFrame({
    'Use Case': [
        'Test Time Forecasting',
        'Yield Prediction',
        'Equipment Downtime',
        'Bin Distribution Trends',
        'Parametric Drift Detection'
    ],
    'Input Features': [
        'Historical test times, device type, test program',
        'Wafer lot data, fab conditions, design changes',
        'Equipment age, maintenance logs, utilization',
        'Historical bin counts, process changes, holidays',
        'Vdd, Idd, freq over time, temp, lot variations'
    ],
    'Prophet Advantage': [
        'Handles test flow changes (changepoints)',
        'Models weekly/monthly fab cycles + holidays',
        'Detects seasonality in equipment failures',
        'Incorporates holiday effects (reduced volume)',
        'Robust to outliers from bad wafers'
    ],
    'Business Impact': [
        'Optimize test capacity planning',
        'Proactive yield management',
        'Preventive maintenance scheduling',
        'Inventory/supply chain optimization',
        'Early detection of process drift'
    ],
    'Forecast Horizon': [
        '1-4 weeks',
        '2-12 weeks',
        '1-8 weeks',
        '2-8 weeks',
        '1-6 weeks'
    ]
})

print('\n🏭 Prophet in Post-Silicon Validation:\n')
print(use_cases.to_string(index=False))

# Visualization
fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Chart 1: Forecast horizon comparison
ax1 = axes[0]
horizons = [4, 12, 8, 8, 6]  # Max weeks
colors_uc = ['#4285F4', '#34A853', '#FBBC04', '#EA4335', '#9C27B0']
bars1 = ax1.barh(use_cases['Use Case'], horizons, color=colors_uc, edgecolor='black', linewidth=1.5)

for i, (bar, horizon) in enumerate(zip(bars1, horizons)):
    ax1.text(horizon + 0.3, i, f'{horizon} weeks', va='center', fontsize=11, fontweight='bold')

ax1.set_xlabel('Forecast Horizon (weeks)', fontsize=12, fontweight='bold')
ax1.set_title('Typical Forecast Horizons for Semiconductor Use Cases', fontsize=14, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)
ax1.set_xlim(0, 14)

# Chart 2: Implementation complexity vs business impact
ax2 = axes[1]
complexity = [3, 4, 3, 2, 4]  # 1-5 scale
impact = [4, 5, 4, 3, 5]  # 1-5 scale

scatter = ax2.scatter(complexity, impact, s=[h*50 for h in horizons],
                     c=colors_uc, alpha=0.6, edgecolors='black', linewidths=2)

for i, label in enumerate(use_cases['Use Case']):
    ax2.annotate(label, (complexity[i], impact[i]),
                xytext=(10, 5), textcoords='offset points',
                fontsize=10, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

ax2.set_xlabel('Implementation Complexity (1=Low, 5=High)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Business Impact (1=Low, 5=High)', fontsize=12, fontweight='bold')
ax2.set_title('Prophet Use Cases: Complexity vs Impact (Bubble Size = Forecast Horizon)',
             fontsize=14, fontweight='bold')
ax2.set_xlim(0.5, 5.5)
ax2.set_ylim(2, 5.5)
ax2.grid(True, alpha=0.3)

# Add quadrants
ax2.axhline(y=3.5, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax2.axvline(x=3.5, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax2.text(1, 5.3, 'Quick Wins', fontsize=11, fontweight='bold', color='green')
ax2.text(4.5, 5.3, 'Strategic', fontsize=11, fontweight='bold', color='blue')
ax2.text(1, 2.2, 'Low Priority', fontsize=11, fontweight='bold', color='gray')
ax2.text(4.5, 2.2, 'Reconsider', fontsize=11, fontweight='bold', color='red')

plt.tight_layout()
plt.savefig('prophet_post_silicon_use_cases.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n✅ Prophet excels at post-silicon forecasting because:')
print('  1. Fab production has weekly/monthly cycles → Prophet models multiple seasonalities')
print('  2. Holidays affect test volume → Prophet has built-in holiday effects')
print('  3. Process changes create trends → Prophet detects changepoints automatically')
print('  4. Test data has outliers (bad lots) → Prophet is robust')
print('  5. Missing data common (equipment downtime) → Prophet handles gaps')
print('\n💡 Start with Yield Prediction or Test Time Forecasting for quick wins!')

## 🎛️ Prophet Hyperparameter Tuning Guide

Key parameters and their effects:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Hyperparameter guide
params = pd.DataFrame({
    'Parameter': [
        'changepoint_prior_scale',
        'seasonality_prior_scale',
        'holidays_prior_scale',
        'seasonality_mode',
        'changepoint_range',
        'n_changepoints',
        'growth'
    ],
    'Default': [0.05, 10.0, 10.0, 'additive', 0.8, 25, 'linear'],
    'Range': [
        '0.001 - 0.5',
        '0.01 - 10.0',
        '0.01 - 10.0',
        'additive/multiplicative',
        '0.5 - 0.95',
        '10 - 50',
        'linear/logistic/flat'
    ],
    'Effect When Increased': [
        'More flexible trend (captures more changepoints)',
        'Stronger seasonal patterns',
        'Larger holiday effects',
        'N/A (categorical)',
        'Changepoints allowed later in series',
        'More potential changepoints',
        'N/A (categorical)'
    ],
    'When to Tune': [
        'Underfit trend → increase; Overfit → decrease',
        'Weak seasonality → increase; Noisy → decrease',
        'Holidays not captured → increase',
        'Seasonality grows with trend → multiplicative',
        'Recent trend changes ignored → increase',
        'Missing trend changes → increase',
        'Saturating growth → logistic; No growth → flat'
    ]
})

print('\n🎛️ Prophet Hyperparameter Tuning Guide:\n')
print(params.to_string(index=False))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Parameter importance (subjective ranking)
ax1 = axes[0, 0]
param_importance = [5, 4, 3, 4, 2, 2, 3]  # 1-5 scale
colors_param = ['red' if imp >= 4 else 'orange' if imp >= 3 else 'yellow' for imp in param_importance]
bars1 = ax1.barh(params['Parameter'], param_importance, color=colors_param, edgecolor='black', linewidth=1.5)

for i, (bar, imp) in enumerate(zip(bars1, param_importance)):
    ax1.text(imp + 0.1, i, f'{imp}/5', va='center', fontsize=10, fontweight='bold')

ax1.set_xlabel('Importance for Tuning (1=Low, 5=High)', fontsize=12, fontweight='bold')
ax1.set_title('Prophet Parameter Importance', fontsize=14, fontweight='bold')
ax1.set_xlim(0, 6)
ax1.grid(axis='x', alpha=0.3)

# Plot 2: Tuning strategy
ax2 = axes[0, 1]
ax2.axis('off')

tuning_steps = [
    "📋 Prophet Tuning Strategy:\n",
    "Step 1: Start with defaults",
    "   • Train baseline model",
    "   • Evaluate on validation set\n",
    "Step 2: Tune growth type",
    "   • Try linear, logistic, flat",
    "   • Check trend saturation\n",
    "Step 3: Tune seasonality mode",
    "   • Additive (constant seasonality)",
    "   • Multiplicative (growing seasonality)\n",
    "Step 4: Adjust changepoint_prior_scale",
    "   • 0.001-0.01: Rigid trend",
    "   • 0.05: Default (balanced)",
    "   • 0.1-0.5: Flexible trend\n",
    "Step 5: Tune seasonality strength",
    "   • Increase seasonality_prior_scale if weak",
    "   • Decrease if too noisy\n",
    "💡 Use cross-validation for final tuning"
]

ax2.text(0.05, 0.95, '\n'.join(tuning_steps), transform=ax2.transAxes,
        fontsize=10, verticalalignment='top', family='monospace',
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3))

# Plot 3: Parameter sensitivity (example values)
ax3 = axes[1, 0]
changepoint_vals = [0.001, 0.01, 0.05, 0.1, 0.5]
mae_vals = [12.5, 10.2, 8.5, 9.1, 15.3]  # Hypothetical MAE

ax3.plot(changepoint_vals, mae_vals, marker='o', linewidth=2, markersize=10,
        color='blue', markerfacecolor='red', markeredgecolor='black', markeredgewidth=2)
min_idx = mae_vals.index(min(mae_vals))
ax3.scatter(changepoint_vals[min_idx], mae_vals[min_idx], s=300, color='gold',
           edgecolors='black', linewidths=3, marker='*', zorder=5, label='Optimal')

ax3.set_xlabel('changepoint_prior_scale', fontsize=12, fontweight='bold')
ax3.set_ylabel('Mean Absolute Error', fontsize=12, fontweight='bold')
ax3.set_title('Example: Tuning Changepoint Prior Scale', fontsize=14, fontweight='bold')
ax3.set_xscale('log')
ax3.grid(True, alpha=0.3)
ax3.legend(fontsize=11)

# Plot 4: Cross-validation approach
ax4 = axes[1, 1]
ax4.axis('off')

cv_code = [
    "💻 Cross-Validation Code:\n",
    "from prophet.diagnostics import cross_validation",
    "from prophet.diagnostics import performance_metrics\n",
    "# Define parameter grid",
    "param_grid = {",
    "    'changepoint_prior_scale': [0.001, 0.01, 0.05, 0.1, 0.5],",
    "    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],",
    "}\n",
    "# Grid search",
    "best_mae = float('inf')",
    "for cp in param_grid['changepoint_prior_scale']:",
    "    for sp in param_grid['seasonality_prior_scale']:",
    "        m = Prophet(changepoint_prior_scale=cp,",
    "                    seasonality_prior_scale=sp)",
    "        m.fit(train_df)",
    "        df_cv = cross_validation(m, initial='730 days',",
    "                                 period='180 days',",
    "                                 horizon='365 days')",
    "        df_p = performance_metrics(df_cv)",
    "        mae = df_p['mae'].mean()",
    "        if mae < best_mae:",
    "            best_mae = mae",
    "            best_params = (cp, sp)\n",
    "💡 Use parallel processing for large grids!"
]

ax4.text(0.02, 0.98, '\n'.join(cv_code), transform=ax4.transAxes,
        fontsize=9, verticalalignment='top', family='monospace',
        bbox=dict(boxstyle='round', facecolor='lightcyan', alpha=0.5))

plt.tight_layout()
plt.savefig('prophet_hyperparameter_tuning.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n✅ Hyperparameter tuning tips:')
print('  1. Start with defaults, tune only if needed')
print('  2. Focus on changepoint_prior_scale and seasonality_mode first')
print('  3. Use cross-validation for objective comparison')
print('  4. Watch for overfitting with high flexibility')
print('  5. Domain knowledge > blind grid search')
print('\n💡 Prophet is designed to work well with minimal tuning!')

## 🎯 Real-World Projects

### Post-Silicon Validation Projects

#### 1. **Long-Horizon Yield Predictor** 💰 $10M+ Capacity Planning
- **Objective:** Forecast weekly yield 26 weeks ahead (±3% MAPE) for 6-month capacity planning
- **Data:** 3+ years weekly yield with process improvements (changepoints), quarterly PM cycles
- **Success Metric:** Beat Holt-Winters by 20% RMSE, automatically detect major process changes
- **Implementation:**
  - Prophet with quarterly seasonality (`add_seasonality(period=13)`)
  - Custom holidays for equipment upgrades (±7 day window)
  - Changepoint prior scale = 0.1 (detect ~5 major improvements)
  - Cross-validate with 8 historical 26-week periods

#### 2. **Multi-Product Yield Forecaster** 💰 $15M+ Mix Optimization
- **Objective:** Forecast yield for 20+ products simultaneously (±5% accuracy) for product mix optimization
- **Data:** Weekly yield per product (50-95%), 104 weeks history, shared PM events
- **Success Metric:** Single pipeline forecasts all products in <2 minutes
- **Implementation:**
  - Loop over 20 products, fit Prophet model per product
  - Shared holidays DataFrame (PM events affect all products)
  - Product-specific seasonality (some products have quarterly patterns, others don't)
  - Aggregate forecasts to compute optimal product mix (maximize total wafers × yield)

#### 3. **Test Time Anomaly Detector** 💰 $3M+ Equipment Health
- **Objective:** Detect test time anomalies 3 days earlier than current methods (90% recall)
- **Data:** Daily average test time (50-80 ms), 365 days history, gradual equipment degradation
- **Success Metric:** Alert when actual deviates from Prophet upper bound by 2σ
- **Implementation:**
  - Prophet with daily + weekly seasonality
  - Fit on 90-day rolling window (adaptive to recent trends)
  - Real-time monitoring: alert if actual > `yhat_upper` for 3 consecutive days
  - Business value: Proactive PM reduces unplanned downtime 50%

#### 4. **Fab-Wide Seasonality Calendar** 💰 $5M+ Planning Tool
- **Objective:** Create automated calendar of seasonal patterns for all 50+ metrics (yield, test time, throughput)
- **Data:** 2+ years of weekly data for 50+ fab metrics
- **Success Metric:** Automatically identify quarterly, monthly, weekly patterns with >80% accuracy
- **Implementation:**
  - Fit Prophet to each metric, extract seasonal components
  - Rank seasonality by amplitude (which metrics have strongest patterns?)
  - Generate visual calendar showing expected seasonal dips/peaks
  - Planning tool: Show expected yield drops for next 12 weeks across all products

---

### General AI/ML Projects

#### 5. **Retail Store Demand Forecasting** 💰 $50M+ Inventory Optimization
- **Objective:** Forecast daily demand for 1000+ stores × 500+ SKUs (±12% MAPE)
- **Data:** 3 years daily sales with holidays (Thanksgiving, Christmas, Black Friday)
- **Success Metric:** Reduce stockouts 40%, overstock 30%
- **Implementation:**
  - Prophet with `add_country_holidays('US')`
  - Custom holidays: store-specific promotions (±3 day window)
  - Yearly + weekly seasonality (Christmas spike, weekend patterns)
  - Parallel processing: forecast 500K store-SKU combinations in <1 hour

#### 6. **Website Traffic Predictor** 💰 $20M+ Server Capacity
- **Objective:** Forecast hourly traffic 72 hours ahead (±15% accuracy) for auto-scaling
- **Data:** 2 years hourly pageviews (10K-1M), daily + weekly + yearly patterns
- **Success Metric:** Auto-scale servers to match forecast, save 30% on cloud costs
- **Implementation:**
  - Prophet with hourly + daily + weekly + yearly seasonality
  - Custom holidays: product launches, marketing campaigns
  - Forecast every hour (rolling), 72-hour horizon
  - Auto-scaling rule: if forecast > 80% current capacity for 6+ hours → scale up

#### 7. **Energy Demand Forecaster (Grid Level)** 💰 $100M+ Grid Optimization
- **Objective:** Forecast hourly electricity demand 48 hours ahead (±5% MAPE) for generator scheduling
- **Data:** 5 years hourly demand (5-50 GW), temperature data, holidays
- **Success Metric:** Optimize generator mix, reduce peak demand charges 20%
- **Implementation:**
  - Prophet with daily + weekly seasonality
  - Add regressor: temperature (demand increases with extreme heat/cold)
  - Holidays: federal holidays (lower industrial demand)
  - Generate 48-hour forecast every 6 hours, update generator schedule

#### 8. **Subscription Churn Forecaster** 💰 $30M+ Retention
- **Objective:** Forecast monthly churn rate 6 months ahead (±10% accuracy) for retention campaigns
- **Data:** 3 years monthly churn rate (2-15%), seasonal patterns (Q1 high due to resolutions)
- **Success Metric:** Plan retention campaigns 3 months ahead, reduce churn 25%
- **Implementation:**
  - Prophet with yearly seasonality (Q1 spike)
  - Trend component captures growth phase effects (early users higher loyalty)
  - Forecast 6 months ahead with 95% confidence intervals
  - Retention budget allocation: $X × forecasted_churn_rate × customer_LTV

---

## ✅ Key Takeaways

### When to Use Prophet

| **Use Case** | **Prophet Advantage** |
|--------------|----------------------|
| Multiple seasonality (daily + weekly + yearly) | Automatic Fourier decomposition, no manual feature engineering |
| Business forecasting (holidays matter) | Built-in holidays for 100+ countries, custom events easy |
| Non-stationary data (trend changes) | Automatic changepoint detection with L1 regularization |
| Missing data | Handles gaps automatically (unlike ARIMA) |
| Large-scale forecasting (100+ series) | Fast parallel fitting, good defaults, minimal tuning |
| Interpretability required | Additive components (trend + seasonality + holidays) easy to explain |

### Advantages ✅
- **Zero manual tuning** (vs ARIMA's p,d,q selection)
- **Handles missing data** automatically
- **Multiple seasonality** (Fourier series, different periods)
- **Changepoint detection** (automatic trend shifts)
- **Holidays and events** (built-in + custom)
- **Fast and scalable** (Stan backend, parallel)
- **Interpretable** (additive components)

### Limitations ❌
- **Linear/logistic trend only** (non-linear trends poorly captured)
- **No multivariate** (can't model multiple correlated series jointly)
- **Additive components** (can't model complex interactions)
- **Requires pandas DataFrame** (specific format: 'ds', 'y')
- **Over-parameterization risk** (too many Fourier terms → overfitting)

### Prophet vs Alternatives

| **Method** | **Best For** | **Avoid When** |
|------------|--------------|----------------|
| **Prophet** | Business forecasting, multiple seasonality, holidays | Need multivariate models, complex non-linear patterns |
| **ARIMA** | Complex autocorrelations, short-memory processes | Non-stationary data, multiple seasonality without SARIMA |
| **Exponential Smoothing** | Simple patterns, fast forecasting | Multiple seasonality, missing data |
| **LSTM/Neural Nets** | Non-linear patterns, multivariate, large data | Need interpretability, small data (<1000 points) |

---

## 🚀 Next Steps
- **Notebook 034**: Advanced time series topics (VAR, state space models)
- **Notebook 036+**: Anomaly detection methods
- **Recommended Practice**: Apply Prophet to real STDF yield data with custom PM holidays
- **Further Reading**: Taylor & Letham (2018) - *Forecasting at Scale* (Prophet paper)